In [3]:
import sys 
sys.path.append('../')

import scipy
import numpy as np
import matplotlib.pyplot as plt
from dataset import numerical_kernel
from utils import *
import torch

In [7]:
idx = 1234
us = scipy.io.loadmat('../data/kernel_estimation/grf1d_N2000_s8193_m0_gamma1_tau1_sigma1_dirichlet.mat')['f']
kernel_types = ['helmholtz'] #['cosine', 'lnabs', 'laplace', 'advection_diffusion', 'helmholtz', 'negative_helmholtz']
for kernel in kernel_types:
    Khh, xh, h = numerical_kernel(13, kernel_type=kernel)
    ws = np.load('../data/kernel_estimation/{:}_8193.npy'.format(kernel))

    sample = torch.tensor(ws[[idx]])
    sample_ref = torch.tensor(Khh[0,0] @ us[idx] * h)[None]

    print('{:} - rl2 : '.format(kernel), rl2_error(sample, sample_ref))

helmholtz - rl2 :  tensor(4.4814e-15, dtype=torch.float64)


/tmp/ipykernel_810239/3531829452.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_ref = torch.tensor(Khh[0,0] @ us[idx] * h)[None]


In [15]:
np.abs(ws).max()

0.019591156594934187

In [13]:
us.min()

-1.8726352221882552